In [12]:
import cv2
import os

# Load query images
query_images = [cv2.imread(f'/Users/Lenovo/Downloads/Homework2/task_5/query/query_{i}.jpg') for i in range(1, 4)]

# Load surveillance images
surveillance_images = []
image_folder = 'task_5/database'
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder, filename))
    if img is not None:
        surveillance_images.append((filename, img))

In [13]:
def preprocess_image(image):
    # Resize image to a standard size (e.g., 256x256)
    resized_image = cv2.resize(image, (256, 256))
    # Convert to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return gray_image

preprocessed_query_images = [preprocess_image(img) for img in query_images]
preprocessed_surveillance_images = [(filename, preprocess_image(img)) for filename, img in surveillance_images]

In [14]:
orb = cv2.ORB_create()

def extract_features(image):
    # Detect keypoints and descriptors using ORB
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

query_features = [extract_features(img) for img in preprocessed_query_images]
surveillance_features = [(filename, extract_features(img)) for filename, img in preprocessed_surveillance_images]

In [15]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

def match_features(query_descriptors, surveillance_descriptors):
    matches = bf.match(query_descriptors, surveillance_descriptors)
    # Sort matches by distance (lower distance is better)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

# Find best matches for each query image
best_matches = {}
for i, (q_kp, q_desc) in enumerate(query_features):
    best_match_filename = None
    best_match_count = 0
    
    for filename, (s_kp, s_desc) in surveillance_features:
        if s_desc is not None:
            matches = match_features(q_desc, s_desc)
            if len(matches) > best_match_count:
                best_match_count = len(matches)
                best_match_filename = filename
    
    best_matches[f'Query Image {i+1}'] = best_match_filename

In [16]:
import matplotlib.pyplot as plt

# Directory where you want to save the result images
output_folder = 'matched_query'

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(3):
    matched_filename = best_matches[f'Query Image {i+1}']
    matched_image = cv2.imread(os.path.join(image_folder, matched_filename))
    
    plt.figure(figsize=(10, 5))
    
    # Plot query image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(query_images[i], cv2.COLOR_BGR2RGB))
    plt.title(f'Query Image {i+1}')
    
    # Plot matched surveillance image
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Matched Surveillance Image: {matched_filename}')
    
    # Save the figure instead of showing it
    output_path = os.path.join(output_folder, f'match_result_{i+1}.png')
    plt.savefig(output_path)
    
    # Close the figure to free memory
    plt.close()